In [4]:
import pandas as pd
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.models import infer_signature
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from sklearn.metrics import roc_auc_score, log_loss, f1_score

# Carregar os datasets
#C:\Users\Anderson\Desktop\REDES_NEURAIS\ENGENHARIA_V2\ENGENHARIA_MACHINE_LEARNING_INFNET\data\raw\dataset_kobe_dev.parquet
dev_dataset_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/raw/dataset_kobe_dev.parquet"
prod_dataset_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/raw/dataset_kobe_prod.parquet"

# Carregar os datasets de desenvolvimento e produção
dev_df = pd.read_parquet(dev_dataset_path)
prod_df = pd.read_parquet(prod_dataset_path)

# Filtrar as colunas relevantes
relevant_columns = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']
dev_df_filtered = dev_df[relevant_columns].copy()
prod_df_filtered = prod_df[relevant_columns].copy()

# Remover linhas com dados faltantes
dev_df_filtered.dropna(inplace=True)
prod_df_filtered.dropna(inplace=True)

# Dividir os dados em conjuntos de treino (80%) e teste (20%) usando uma escolha aleatória e estratificada
dev_train, dev_test = train_test_split(dev_df_filtered, test_size=0.2, random_state=42, stratify=dev_df_filtered['shot_made_flag'])
prod_train, prod_test = train_test_split(prod_df_filtered, test_size=0.2, random_state=42, stratify=prod_df_filtered['shot_made_flag'])
    
# Iniciar uma nova corrida no MLflow
with mlflow.start_run(run_name="PipelineAplicacao"):
    # Calcular as proporções de treino e teste
    dev_train_ratio = len(dev_train) / len(dev_df_filtered)
    dev_test_ratio = len(dev_test) / len(dev_df_filtered)
    prod_train_ratio = len(prod_train) / len(prod_df_filtered)
    prod_test_ratio = len(prod_test) / len(prod_df_filtered)

    # Registrar os parâmetros e métricas no MLflow
    mlflow.log_param("Porcentagem_de_Teste_Dev", dev_test_ratio)
    mlflow.log_param("Porcentagem_de_Treino_Dev", dev_train_ratio)
    mlflow.log_param("Porcentagem_de_Teste_Prod", prod_test_ratio)
    mlflow.log_param("Porcentagem_de_Treino_Prod", prod_train_ratio)
    mlflow.log_metric("Tamanho_do_Conjunto_de_Treino_Dev", len(dev_train))
    mlflow.log_metric("Tamanho_do_Conjunto_de_Teste_Dev", len(dev_test))
    mlflow.log_metric("Tamanho_do_Conjunto_de_Treino_Prod", len(prod_train))
    mlflow.log_metric("Tamanho_do_Conjunto_de_Teste_Prod", len(prod_test))
    
    # Setup dos dados para treinamento usando PyCaret
    setup_params = {
        "target": "shot_made_flag",
        "session_id": 42
    }
    
    setup(**setup_params)
    # Treinar modelos de classificação usando PyCaret
    automl_model = automl(optimize="AUC")
    
    # Salvar o melhor modelo como um arquivo .pkl
    save_model(automl_model, "best_model_pycaret")
    
    # Fazer previsões nos dados de teste com o melhor modelo
    predictions = predict_model(automl_model, data=dev_test)

    # Log das métricas
    mlflow.log_metric("AUC", roc_auc_score(predictions['shot_made_flag'], predictions['prediction_label']))
    mlflow.log_metric("Log Loss", log_loss(predictions['shot_made_flag'], predictions['prediction_label']))
    mlflow.log_metric("F1 Score", f1_score(predictions['shot_made_flag'], predictions['prediction_label']))

    # Salvar o modelo no MLflow
    mlflow.sklearn.log_model(automl_model, "best_model")

    # Registrar o modelo para produção
    model_uri = f"file:///C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/models/best_model"
    mlflow.register_model(model_uri, "best_model_production")

    # Transicionar o modelo para o estágio de produção
    client = MlflowClient()
    client.transition_model_version_stage(
        name="best_model_production",
        version=1, 
        stage="Production",
        archive_existing_versions=True
    )


ValueError: One and only one of data and data_func must be set

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss

# Dados de produção
predicted_probs_prod = [0.5, 0.5, 0.5, 0.5, 0.5]  # Substitua pelos valores reais
real_results_prod = [0, 1, 1, 0, 1]  # Substitua pelos valores reais

# Dados de desenvolvimento
predicted_probs_dev = [0.5, 0.5, 0.5, 0.731059, 0.5]  # Substitua pelos valores reais
real_results_dev = [0, 1, 1, 0, 1]  # Substitua pelos valores reais

# Calcular as métricas para os dados de produção
accuracy_prod = accuracy_score(real_results_prod, [1 if p >= 0.5 else 0 for p in predicted_probs_prod])
precision_prod = precision_score(real_results_prod, [1 if p >= 0.5 else 0 for p in predicted_probs_prod])
recall_prod = recall_score(real_results_prod, [1 if p >= 0.5 else 0 for p in predicted_probs_prod])
f1_prod = f1_score(real_results_prod, [1 if p >= 0.5 else 0 for p in predicted_probs_prod])
logloss_prod = log_loss(real_results_prod, predicted_probs_prod)

# Calcular as métricas para os dados de desenvolvimento
accuracy_dev = accuracy_score(real_results_dev, [1 if p >= 0.5 else 0 for p in predicted_probs_dev])
precision_dev = precision_score(real_results_dev, [1 if p >= 0.5 else 0 for p in predicted_probs_dev])
recall_dev = recall_score(real_results_dev, [1 if p >= 0.5 else 0 for p in predicted_probs_dev])
f1_dev = f1_score(real_results_dev, [1 if p >= 0.5 else 0 for p in predicted_probs_dev])
logloss_dev = log_loss(real_results_dev, predicted_probs_dev)

# Imprimir as métricas
print("Métricas para os dados de produção:")
print(f"Acurácia: {accuracy_prod}")
print(f"Precisão: {precision_prod}")
print(f"Recall: {recall_prod}")

print(f"F1-Score: {f1_prod}")
print(f"Log Loss: {logloss_prod}")
print("\nMétricas para os dados de desenvolvimento:")
print(f"Acurácia: {accuracy_dev}")
print(f"Precisão: {precision_dev}")
print(f"Recall: {recall_dev}")
print(f"F1-Score: {f1_dev}")
print(f"Log Loss: {logloss_dev}")



Métricas para os dados de produção:
Acurácia: 0.6
Precisão: 0.6
Recall: 1.0
F1-Score: 0.7499999999999999
Log Loss: 0.6931471805599453

Métricas para os dados de desenvolvimento:
Acurácia: 0.6
Precisão: 0.6
Recall: 1.0
F1-Score: 0.7499999999999999
Log Loss: 0.8171703953063254


In [32]:
import pandas as pd

# Carregar os dados de produção
prod_data_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/code/predictions_prod_2024-05-09_22-28-49.parquet"
prod_data = pd.read_parquet(prod_data_path)

# Carregar os dados de desenvolvimento
dev_data_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/code/predictions_dev_2024-05-09_22-34-50.parquet"
dev_data = pd.read_parquet(dev_data_path)

# Visualizar os dados
print("Dados de produção:")
print(prod_data.head())

print("\nDados de desenvolvimento:")
print(dev_data.head())


Dados de produção:
   probability
0          0.5
1          0.5
2          0.5
3          0.5
4          0.5

Dados de desenvolvimento:
   probability
0     0.500000
1     0.500000
2     0.500000
3     0.731059
4     0.500000


In [28]:
import mlflow
import pandas as pd
from sklearn.metrics import log_loss, f1_score
from pycaret.classification import load_model
from datetime import datetime
import numpy as np

def load_production_data(file_path):
    # Carregar a base de produção
    prod_data = pd.read_parquet(file_path)
    return prod_data

def load_dev_data(file_path):
    # Carregar a base de desenvolvimento
    dev_data = pd.read_parquet(file_path)
    return dev_data

def apply_model(model, data):
    # Aplicar o modelo aos dados e obter as pontuações
    scores = model.predict(data)
    # Aplicar função sigmoid para transformar em probabilidades
    probabilities = 1 / (1 + np.exp(-scores))
    return probabilities

def evaluate_model(model, X, y):
    # Avaliar o modelo com os dados
    y_pred = model.predict(X)
    logloss = log_loss(y, y_pred)
    f1 = f1_score(y, y_pred)
    return logloss, f1

def compare_datasets(dev_data, prod_data):
    # Comparar resultados do modelo entre os conjuntos de dados
    logloss_dev, f1_dev = evaluate_model(model, dev_data.drop(columns=['shot_made_flag']), dev_data['shot_made_flag'])
    logloss_prod, f1_prod = evaluate_model(model, prod_data.drop(columns=['shot_made_flag']), prod_data['shot_made_flag'])

    print("Resultados do modelo para os dados de desenvolvimento:")
    print("Log Loss:", logloss_dev)
    print("F1 Score:", f1_dev)
    print()

    print("Resultados do modelo para os dados de produção:")
    print("Log Loss:", logloss_prod)
    print("F1 Score:", f1_prod)

if __name__ == "__main__":
    # Carregar o modelo treinado
    model_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/code/mlruns/216971763806645027/4dbe79efd23f4d4984b0522e3ad0bd46/artifacts/model/model"
    model = load_model(model_path)

    # Carregar os dados de produção
    production_data_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/processed/data_filtered_prod.parquet"
    prod_data = load_production_data(production_data_path)

    # Carregar os dados de desenvolvimento
    dev_data_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/processed/data_filtered_dev.parquet"
    dev_data = load_dev_data(dev_data_path)

    # Comparar resultados do modelo entre os conjuntos de dados
    compare_datasets(dev_data, prod_data)


Transformation Pipeline and Model Successfully Loaded
Resultados do modelo para os dados de desenvolvimento:
Log Loss: 6.7659844798136985
F1 Score: 0.5223325750430006

Resultados do modelo para os dados de produção:
Log Loss: 5.252267767958857
F1 Score: 0.0


In [25]:
import mlflow
import pandas as pd
from sklearn.metrics import log_loss, f1_score
from pycaret.classification import load_model
from datetime import datetime
import numpy as np

def load_production_data(file_path):
    # Carregar a base de produção
    prod_data = pd.read_parquet(file_path)
    return prod_data

def load_dev_data(file_path):
    # Carregar a base de desenvolvimento
    dev_data = pd.read_parquet(file_path)
    return dev_data

def apply_model(model, data):
    # Aplicar o modelo aos dados e obter as pontuações
    scores = model.predict(data)
    # Aplicar função sigmoid para transformar em probabilidades
    probabilities = 1 / (1 + np.exp(-scores))
    return probabilities

def evaluate_model(model, X, y):
    # Avaliar o modelo com os dados
    y_pred = model.predict(X)
    logloss = log_loss(y, y_pred)
    f1 = f1_score(y, y_pred)
    return logloss, f1

if __name__ == "__main__":
    # Carregar o modelo treinado
    model_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/code/mlruns/216971763806645027/4dbe79efd23f4d4984b0522e3ad0bd46/artifacts/model/model"
    model = load_model(model_path)

    # Carregar os dados de produção
    production_data_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/processed/data_filtered_prod.parquet"
    prod_data = load_production_data(production_data_path)

    # Carregar os dados de desenvolvimento
    dev_data_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/processed/data_filtered_dev.parquet"
    dev_data = load_dev_data(dev_data_path)

    # Aplicar o modelo aos dados de produção
    predictions_prod = apply_model(model, prod_data.drop(columns=['shot_made_flag']))

    # Avaliar o modelo com os dados de produção
    logloss_prod, f1_prod = evaluate_model(model, prod_data.drop(columns=['shot_made_flag']), prod_data['shot_made_flag'])

    # Aplicar o modelo aos dados de desenvolvimento
    predictions_dev = apply_model(model, dev_data.drop(columns=['shot_made_flag']))

    # Avaliar o modelo com os dados de desenvolvimento
    logloss_dev, f1_dev = evaluate_model(model, dev_data.drop(columns=['shot_made_flag']), dev_data['shot_made_flag'])

    # Nomear a rodada do MLflow
    mlflow.set_experiment("PipelineAplicacao")
    with mlflow.start_run(run_name="PipelineAplicacao"):
        # Salvar resultados como artefatos
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        
        # Converter os arrays numpy em dataframes do pandas
        predictions_prod_df = pd.DataFrame(predictions_prod, columns=['probability'])
        predictions_dev_df = pd.DataFrame(predictions_dev, columns=['probability'])

        # Salvar os dataframes como arquivos parquet
        predictions_prod_df.to_parquet(f"predictions_prod_{timestamp}.parquet")
        predictions_dev_df.to_parquet(f"predictions_dev_{timestamp}.parquet")
        
        # Logar as métricas de produção
        mlflow.log_metric("log_loss_prod", logloss_prod)
        mlflow.log_metric("f1_score_prod", f1_prod)

        # Logar as métricas de desenvolvimento
        mlflow.log_metric("log_loss_dev", logloss_dev)
        mlflow.log_metric("f1_score_dev", f1_dev)


Transformation Pipeline and Model Successfully Loaded


In [14]:
import mlflow

# Listar todas as corridas
runs = mlflow.search_runs()

# Filtrar as corridas pelo nome do experimento
runs_modelobom = runs[runs['tags.mlflow.runName'] == "MODELOBOM"]

if not runs_modelobom.empty:
    # Se houver corridas com o nome "MODELOBOM", obtemos o ID da primeira corrida
    run_id = runs_modelobom.iloc[0]['run_id']
    
    # Obter informações sobre a corrida MODELOBOM
    run_info = mlflow.get_run(run_id)
    
    # Acessar os artefatos (modelos) diretamente
    artifacts = run_info.to_dictionary()['info']['artifact_uri']
    
    if artifacts:
        print("A corrida MODELOBOM foi registrada e contém os seguintes artefatos (modelos):")
        print(artifacts)
    else:
        print("A corrida MODELOBOM foi registrada, mas não contém modelos.")
else:
    print("A corrida MODELOBOM não foi registrada no MLflow.")


A corrida MODELOBOM foi registrada e contém os seguintes artefatos (modelos):
file:///c:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/code/mlruns/216971763806645027/4dbe79efd23f4d4984b0522e3ad0bd46/artifacts


In [4]:
import mlflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, f1_score
from pycaret.classification import setup, create_model

#  MLflow
mlflow.set_experiment("Treinamento")

#  treinamento e teste filtrados
dev_processed_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/processed/data_filtered_dev.parquet"
prod_processed_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/processed/data_filtered_prod.parquet"
dev_df = pd.read_parquet(dev_processed_path)
prod_df = pd.read_parquet(prod_processed_path)

# Separar os dados filtrados para treinamento e teste
X_dev = dev_df.drop(columns=['shot_made_flag'])
y_dev = dev_df['shot_made_flag']
X_train, X_test, y_train, y_test = train_test_split(X_dev, y_dev, test_size=0.2, stratify=y_dev, random_state=42)

# Verificar se uma corrida está ativa
if mlflow.active_run():
    mlflow.end_run()

# Iniciar uma nova corrida do MLflow
with mlflow.start_run(run_name="Treinamento"):

    # Treinar o modelo de regressão logística
    with mlflow.start_run(run_name="Logistic Regression", nested=True):
        lr_model = create_model('lr', verbose=False)
        lr_pred_test = lr_model.predict_proba(X_test)
        lr_logloss = log_loss(y_test, lr_pred_test)
        mlflow.log_param("model", "Logistic Regression")
        mlflow.log_metric("log_loss", lr_logloss)
        print(f"Métricas para Regressão Logística:\nLog Loss: {lr_logloss}")

    #  árvore de decisão
    with mlflow.start_run(run_name="Decision Tree", nested=True):
        tree_model = create_model('dt', verbose=False)
        tree_pred_test = tree_model.predict_proba(X_test)
        tree_logloss = log_loss(y_test, tree_pred_test)
        tree_f1 = f1_score(y_test, tree_model.predict(X_test))
        mlflow.log_param("model", "Decision Tree")
        mlflow.log_metric("log_loss", tree_logloss)
        mlflow.log_metric("f1_score", tree_f1)
        print(f"Métricas para Árvore de Decisão:\nLog Loss: {tree_logloss}\nF1 Score: {tree_f1}")

    # Selecionar o modelo com menor log loss para finalização
    if lr_logloss < tree_logloss:
        best_model = lr_model
        best_model_name = "Regressão Logística"
    else:
        best_model = tree_model
        best_model_name = "Árvore de Decisão"

    print(f"O melhor modelo é: {best_model_name}")

    # Encerrar a corrida atual do MLflow
    mlflow.end_run()

    # Iniciar uma nova corrida para salvar o melhor modelo
    with mlflow.start_run(run_name="Melhor Modelo"):

        # Log do modelo
        mlflow.sklearn.log_model(best_model, "model")

        # Log dos parâmetros do modelo
        mlflow.log_param("model_name", best_model_name)

        # Log das métricas
        mlflow.log_metric("log_loss", min(lr_logloss, tree_logloss))
        if best_model_name == "Árvore de Decisão":
            mlflow.log_metric("f1_score", tree_f1)

2024/05/09 21:22:12 INFO mlflow.tracking.fluent: Experiment with name 'Treinamento' does not exist. Creating a new experiment.


Métricas para Regressão Logística:
Log Loss: 0.6783615727270217
Métricas para Árvore de Decisão:
Log Loss: 4.3672161877959015
F1 Score: 0.8137086903304773
O melhor modelo é: Regressão Logística


In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import setup
import mlflow

if mlflow.active_run():
    mlflow.end_run()
#  MLflow
mlflow.set_experiment("PreparacaoDados")

# Iniciar novo  MLflow
with mlflow.start_run(run_name="PreparacaoDados"):

    # C treinamento e teste
    dev_data_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/raw/dataset_kobe_dev.parquet"
    prod_data_path = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/raw/dataset_kobe_prod.parquet"
    dev_df = pd.read_parquet(dev_data_path)
    prod_df = pd.read_parquet(prod_data_path)

    # colunas necessárias
    selected_columns = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']
    dev_df = dev_df[selected_columns]
    prod_df = prod_df[selected_columns]

    # Remover linhas com dados faltantes
    dev_df.dropna(inplace=True)
    prod_df.dropna(inplace=True)

    # Salvar o dataset resultante
    processed_data_dir = "C:/Users/Anderson/Desktop/REDES_NEURAIS/ENGENHARIA_V2/ENGENHARIA_MACHINE_LEARNING_INFNET/data/processed"
    os.makedirs(processed_data_dir, exist_ok=True)
    dev_processed_path = os.path.join(processed_data_dir, "data_filtered_dev.parquet")
    prod_processed_path = os.path.join(processed_data_dir, "data_filtered_prod.parquet")
    dev_df.to_parquet(dev_processed_path)
    prod_df.to_parquet(prod_processed_path)

    # Separar os dados em treino e teste (80% treino, 20% teste)
    X_dev = dev_df.drop(columns=['shot_made_flag'])
    y_dev = dev_df['shot_made_flag']
    X_train, X_test, y_train, y_test = train_test_split(X_dev, y_dev, test_size=0.2, stratify=y_dev, random_state=42)

    # Salvar os datasets de treino e teste
    train_data_path = os.path.join(processed_data_dir, "base_train.parquet")
    test_data_path = os.path.join(processed_data_dir, "base_test.parquet")
    X_train.to_parquet(train_data_path)
    y_train.to_frame().to_parquet(train_data_path.replace('.parquet', '_target.parquet'))
    X_test.to_parquet(test_data_path)
    y_test.to_frame().to_parquet(test_data_path.replace('.parquet', '_target.parquet'))

    # Verificar se a coluna shot_made_flag está presente nos dados de treinamento e teste
    print("Colunas presentes nos dados de treinamento:")
    print(X_train.columns)
    print("Colunas presentes nos dados de teste:")
    print(X_test.columns)

    # Configurar o ambiente de classificação do PyCaret
    clf_setup = setup(data=dev_df, target='shot_made_flag')

    # Registar os parâmetros no MLflow
    mlflow.log_param("test_size_percent", 0.2)


Colunas presentes nos dados de treinamento:
Index(['lat', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'shot_distance'],
      dtype='object')
Colunas presentes nos dados de teste:
Index(['lat', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'shot_distance'],
      dtype='object')


,Description,Value
0,Session id,241
1,Target,shot_made_flag
2,Target type,Binary
3,Original data shape,"(20285, 7)"
4,Transformed data shape,"(20285, 7)"
5,Transformed train set shape,"(14199, 7)"
6,Transformed test set shape,"(6086, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple
